In [18]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(rstatix)
library(data.table)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')


Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter




In [6]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Figure5/01_Frequency_Comparison/selected_samples_with_acutal_flu_year.csv")

In [7]:
file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data$pbmc_sample_id,".csv")

df_list<-read_pseudobulk_expression(file_list)

[1] "Total reading time: 32.976 seconds"
[1] "The length of the list matches the length of the input path."


In [8]:
genes_of_interest <- c( "IGHG1", "IGHG2","IGHG3", "IGHG4","IGHM","IGHD")



In [9]:
length(genes_of_interest)

[1] 6

In [10]:
  
celltype_list <- lapply(df_list, function(df) { df <- df[genes_of_interest, grep('Core memory B cell', names(df), fixed = TRUE),drop=FALSE] })


In [11]:
exp_matrix<-do.call(cbind,celltype_list)

colnames(exp_matrix)<-sub(":.*", "", colnames(exp_matrix))
rownames(meta_data)<-meta_data$pbmc_sample_id

In [12]:
meta_data<-meta_data %>% filter(Flu_Year=='2020-2021',Flu_Day=='Day 7')

In [13]:
exp_matrix<-exp_matrix[meta_data$pbmc_sample_id]

In [14]:
    exp_matrix$RowNames <- rownames(exp_matrix)
    long_format <- melt(exp_matrix, id.vars = "RowNames")
    colnames(long_format) <- c('Gene', 'pbmc_sample_id', 'Mean_Expression')


Warning message:
“The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is superseded and is no longer actively developed, and this redirection is now deprecated. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace, i.e. reshape2::melt(exp_matrix). In the next version, this warning will become an error.”


In [15]:
long_format<-left_join(long_format,meta_data) 

Joining with `by = join_by(pbmc_sample_id)`


In [16]:
colnames(long_format)

[1] "Gene"                                     
 [2] "pbmc_sample_id"                           
 [3] "Mean_Expression"                          
 [4] "X.1"                                      
 [5] "X"                                        
 [6] "Unnamed..0"                               
 [7] "lastUpdated"                              
 [8] "sample.id"                                
 [9] "sample.bridgingControl"                   
[10] "sample.sampleKitGuid"                     
[11] "sample.visitName"                         
[12] "sample.visitDetails"                      
[13] "sample.drawDate"                          
[14] "sample.daysSinceFirstVisit"               
[15] "file.id"                                  
[16] "file.name"                                
[17] "file.batchID"                             
[18] "file.panel"                               
[19] "file.pool"                                
[20] "file.fileType"                            
[21] "file.majorVersion"                        
[22] "subject.id"                               
[23] "subject.biologicalSex"                    
[24] "subject.birthYear"                        
[25] "subject.ethnicity"                        
[26] "subject.partnerCode"                      
[27] "subject.race"                             
[28] "subject.subjectGuid"                      
[29] "cohort.cohortGuid"                        
[30] "sample.diseaseStatesRecordedAtVisit"      
[31] "subject.covidVaxDose1.daysSinceFirstVisit"
[32] "subject.covidVaxDose2.daysSinceFirstVisit"
[33] "Covid_exclusion"                          
[34] "subjectGuid"                              
[35] "CMV"                                      
[36] "Flu_Year"                                 
[37] "Flu_Day"

In [19]:
long_format %>% group_by(Gene)%>% wilcox_test(Mean_Expression~cohort.cohortGuid)

,Gene,.y.,group1,group2,n1,n2,statistic,p
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,IGHD,Mean_Expression,BR1,BR2,26,42,641,0.235
2,IGHG1,Mean_Expression,BR1,BR2,26,42,570,0.768
3,IGHG2,Mean_Expression,BR1,BR2,26,42,612,0.411
4,IGHG3,Mean_Expression,BR1,BR2,26,42,513,0.684
5,IGHG4,Mean_Expression,BR1,BR2,26,42,581,0.665
6,IGHM,Mean_Expression,BR1,BR2,26,42,632,0.283


In [20]:
write.csv(long_format,"IGHG1_IGHG2_CoreB.csv")